## Corriente de electrones en sistemas abiertos e impulsados de dos puntos cuánticos 

In [1]:
from joblib import delayed, Parallel

from tqdm.auto import tqdm
import numpy as np

import qutip as qt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

%matplotlib widget

El Hamiltoniano del sistema en la base $\mathscr{B}=\{\ket{\varnothing}, \ket{L}, \ket{R}\}$:

$H= \begin{pmatrix}
        0 & 0 & 0 \\
        0 & \delta/2 & \tau/2 \\
        0 & \tau/2 & -\delta/2
    \end{pmatrix}$
    
donde $\delta(t)=(\delta_0+A\cos{wt})$

In [2]:
# parámetros y hamiltoniano
w = 1
tau = 0.05*w

H0 = qt.Qobj(np.array([(0,0,0),(0,0,1), (0,1,0)]))
H1 = qt.Qobj(np.array([(0,0,0),(0,1/2,0), (0,0,-1/2)]))

In [3]:
E = qt.basis(3, 0)
L = qt.basis(3, 1)
R = qt.basis(3, 2)

gammaL = 0.1*tau
gammaR = 0.1*tau

cL = qt.Qobj(np.array([(0,1,0),(0,0,0),(0,0,0)]))
cR = qt.Qobj(np.array([(0,0,1),(0,0,0),(0,0,0)]))

LL = np.sqrt(gammaL)*cL
LR = np.sqrt(gammaR)*cR

c_ops = [qt.dag(LL),LR]

In [4]:
# rangos de los parámetros del sistema
delta0s = np.linspace(-5*w, 5*w, 201)
As = np.linspace(0, 10*w, 200)

In [5]:
def H1_args(t, args):
    A = args['A']
    delta0 = args['delta0']
    
    return delta0 + A * np.cos(w*t)

In [6]:
H = [tau*H0, [H1, H1_args]]
T = 2*np.pi/w
T_list=np.linspace(0, T, 1000)

In [7]:
# función que calcula un píxel de la imagen final 
def pixel(delta0, A):
    args = {'A': A, 'delta0': delta0}
    U = qt.propagator(H, T, c_ops, args)
    rho_ss = qt.propagator_steadystate(U)
    rho_ss = np.array(qt.mesolve(H, rho_ss, T_list, c_ops,args=args).states)
    rho_ss = np.array([rho_i[:] for rho_i in rho_ss])

    pR=(np.mean(rho_ss[:, 2, 2]).real)
    return pR

In [8]:
# pintamos la corriente normalizada (I=rho_RR) para distintos valores del detuning y de tau
deltas_mat = []
As_mat = []

for i in range(len(delta0s)):
     for j in range(len(As)):
        As_mat.append(As[j])
        deltas_mat.append(delta0s[i])

In [9]:
pbar = tqdm(range(len(deltas_mat)), desc='Calculando...')
resultados = Parallel(n_jobs=-1)(delayed(pixel)(deltas_mat[i], As_mat[i]) for i in pbar)
np.save('current_dqd_driven.npy', resultados)

Calculando...:   0%|          | 0/40200 [00:00<?, ?it/s]

In [10]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors

def hex_to_rgba(hex_color):
    """
    Convierte un color en formato hexadecimal a formato RGBA.
    
    Args:
    - hex_color: Color en formato hexadecimal.
    
    Returns:
    - Color en formato RGBA.
    """
    rgb = mcolors.hex2color(hex_color)
    return (*rgb, 1)

def create_custom_cmap(colors, positions):
    """
    Crea un colormap personalizado a partir de una lista de colores y sus posiciones.
    
    Args:
    - colors: Lista de colores en formato hexadecimal.
    - positions: Lista de posiciones normalizadas (entre 0 y 1).
    
    Returns:
    - Colormap personalizado.
    """
    if len(colors) != len(positions):
        raise ValueError("La longitud de 'colors' debe ser igual a la longitud de 'positions'.")
    
    rgba_colors = [hex_to_rgba(color) for color in colors]
    
    cmap_dict = {'red': [], 'green': [], 'blue': []}
    
    for pos, color in zip(positions, rgba_colors):
        for i, channel in enumerate(['red', 'green', 'blue']):
            cmap_dict[channel].append((pos, color[i], color[i]))
    
    return LinearSegmentedColormap('custom_colormap', cmap_dict)

# Definir lista de colores en formato hexadecimal y posiciones
hex_colors = ['w','#C1CFF6','#5F9EA0','#4D8283','#537E83','#DF9595','#E19B9B', '#F17F8A','#F13D70']
positions = [0.0, 0.05, 0.12,0.2,0.28,0.53,0.57,0.72,1]  # Normalizado entre 0 y 1

# Crear el colormap personalizado
custom_cmap = create_custom_cmap(hex_colors, positions)

In [11]:
resultados = np.load('current_dqd_driven.npy')

In [12]:
resultados_mat = np.array(resultados).reshape((len(delta0s), len(As)))

fig, ax = plt.subplots(figsize = (4,4))
im = ax.imshow(resultados_mat.T, cmap = custom_cmap, origin='lower', aspect='auto',
                interpolation='none', extent=[min(delta0s), max(delta0s), min(As), max(As)])
plt.xlabel(r'$\delta_0/\omega$')
plt.ylabel(r'$A/\omega$')
plt.title('(a)')
cbar = plt.colorbar(im)
cbar.ax.set_ylabel(r'I$/(e\Gamma_R)$')
plt.tight_layout()

plt.savefig('current_dqd_driv.pdf')